In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import os
from scipy import stats
from pathlib import Path
import re
from IPython.display import HTML, display
import pdfkit  # New import for PDF conversion
from jinja2 import Template
import base64
from io import BytesIO
import kaleido

def create_scrollable_container():
    """Create a scrollable container for wide plots"""
    scroll_css = """
        <style>
            .plotly-container {
                width: 100%;
                overflow-x: auto;
                overflow-y: hidden;
                padding: 20px 0;
            }
        </style>
    """
    return HTML(scroll_css)

# Display the CSS
display(create_scrollable_container())

def extract_metadata_from_filename(filename):
    """Extract metadata from filename"""
    match = re.match(r'(Cal|Lin|Fin|Rot)_(\w+)_L([lmh])T(\d)\.html', filename)
    if match:
        task_type, subject, difficulty, trial = match.groups()
        return {
            'task_type': task_type,
            'subject': subject,
            'difficulty': difficulty,
            'trial': trial
        }
    return None

def calculate_statistics(metrics_data):
    """Calculate statistical summaries"""
    stats_dict = {
        'attention': {
            'mean': np.mean(metrics_data['attention']['values']),
            'std': np.std(metrics_data['attention']['values']),
            'median': np.median(metrics_data['attention']['values']),
            'max': np.max(metrics_data['attention']['values']),
            'min': np.min(metrics_data['attention']['values'])
        },
        'meditation': {
            'mean': np.mean(metrics_data['meditation']['values']),
            'std': np.std(metrics_data['meditation']['values']),
            'median': np.median(metrics_data['meditation']['values']),
            'max': np.max(metrics_data['meditation']['values']),
            'min': np.min(metrics_data['meditation']['values'])
        },
        'performance': {
            'accuracy': np.mean(metrics_data['correct_ans']) * 100,
            'mean_rt': np.mean(metrics_data['response_times']),
            'std_rt': np.std(metrics_data['response_times']),
            'median_rt': np.median(metrics_data['response_times'])
        }
    }
    return stats_dict


def show_dashboard(fig):
    """Display the dashboard in a scrollable container"""
    dashboard_html = f"""
        <div class="plotly-container">
            {fig.to_html(full_html=False, include_plotlyjs='cdn')}
        </div>
    """
    display(HTML(dashboard_html))


def create_dashboard(folder_path):
    """Create comprehensive dashboard with all visualizations and statistics"""
    # Get list of all visualization files
    vis_path = os.path.join(folder_path, 'visualizations')
    files = [f for f in sorted(os.listdir(vis_path)) if f.endswith('.html')]
    
    # Create data structures for statistics
    all_data = {
        'task_type': [],
        'difficulty': [],
        'subject': [],
        'trial': [],
        'attention_mean': [],
        'meditation_mean': [],
        'accuracy': [],
        'response_time_mean': [],
        'signal_quality_mean': []
    }
    
    # Process each file
    for file in files:
        metadata = extract_metadata_from_filename(file)
        if metadata:
            # Add metadata to data structure
            all_data['task_type'].append(metadata['task_type'])
            all_data['difficulty'].append(metadata['difficulty'])
            all_data['subject'].append(metadata['subject'])
            all_data['trial'].append(metadata['trial'])
            
            # Load and process data (you'll need to modify this based on your data structure)
            # For now, adding placeholder values
            all_data['attention_mean'].append(np.random.normal(60, 10))
            all_data['meditation_mean'].append(np.random.normal(55, 15))
            all_data['accuracy'].append(np.random.normal(75, 15))
            all_data['response_time_mean'].append(np.random.normal(1.5, 0.3))
            all_data['signal_quality_mean'].append(np.random.normal(90, 5))
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Create dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Attention by Task Type and Difficulty',
            'Meditation by Task Type and Difficulty',
            'Accuracy by Task Type',
            'Response Times by Task Type',
            'Performance Summary',
            'Statistical Summary'
        ),
        specs=[[{'type': 'box'}, {'type': 'box'}],
               [{'type': 'box'}, {'type': 'box'}],
               [{'type': 'table'}, {'type': 'table'}]]
    )
    
    # Plot 1: Attention boxplot
    for task in df['task_type'].unique():
        for diff in ['l', 'm', 'h']:
            mask = (df['task_type'] == task) & (df['difficulty'] == diff)
            fig.add_trace(
                go.Box(
                    y=df[mask]['attention_mean'],
                    name=f'{task}-{diff}',
                    boxmean=True
                ),
                row=1, col=1
            )
    
    # Plot 2: Meditation boxplot
    for task in df['task_type'].unique():
        for diff in ['l', 'm', 'h']:
            mask = (df['task_type'] == task) & (df['difficulty'] == diff)
            fig.add_trace(
                go.Box(
                    y=df[mask]['meditation_mean'],
                    name=f'{task}-{diff}',
                    boxmean=True
                ),
                row=1, col=2
            )
    
    # Plot 3: Accuracy boxplot
    for task in df['task_type'].unique():
        mask = df['task_type'] == task
        fig.add_trace(
            go.Box(
                y=df[mask]['accuracy'],
                name=task,
                boxmean=True
            ),
            row=2, col=1
        )
    
    # Plot 4: Response Times boxplot
    for task in df['task_type'].unique():
        mask = df['task_type'] == task
        fig.add_trace(
            go.Box(
                y=df[mask]['response_time_mean'],
                name=task,
                boxmean=True
            ),
            row=2, col=2
        )
    
    # Calculate summary statistics
    summary_stats = df.groupby('task_type').agg({
        'accuracy': ['mean', 'std'],
        'response_time_mean': ['mean', 'std'],
        'attention_mean': ['mean', 'std'],
        'meditation_mean': ['mean', 'std']
    }).round(2)
    
    # Add summary table
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Task Type', 'Accuracy (%)', 'Response Time (s)', 
                       'Attention', 'Meditation'],
                font=dict(size=12),
                align='left'
            ),
            cells=dict(
                values=[
                    summary_stats.index,
                    summary_stats['accuracy']['mean'],
                    summary_stats['response_time_mean']['mean'],
                    summary_stats['attention_mean']['mean'],
                    summary_stats['meditation_mean']['mean']
                ],
                font=dict(size=11),
                align='left'
            )
        ),
        row=3, col=1
    )
    
    # Perform statistical tests
    task_types = df['task_type'].unique()
    difficulty_levels = ['l', 'm', 'h']
    
    stats_results = []
    for metric in ['attention_mean', 'meditation_mean', 'accuracy']:
        for task in task_types:
            task_data = df[df['task_type'] == task][metric]
            # ANOVA between difficulty levels
            diff_groups = [df[(df['task_type'] == task) & (df['difficulty'] == diff)][metric] 
                         for diff in difficulty_levels]
            f_stat, p_val = stats.f_oneway(*diff_groups)
            stats_results.append([
                f"{task}-{metric}",
                "Difficulty ANOVA",
                f"F={f_stat:.2f}",
                f"p={p_val:.3f}"
            ])
    
    # Add statistical results table
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Comparison', 'Test', 'Statistic', 'p-value'],
                font=dict(size=12),
                align='left'
            ),
            cells=dict(
                values=list(zip(*stats_results)),
                font=dict(size=11),
                align='left'
            )
        ),
        row=3, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1200,
        width=1600,
        title_text="Mental Workload Analysis Dashboard",
        showlegend=True,
        boxmode='group',
        margin=dict(l=50, r=50, t=100, b=50),  # Adjust margins
        # Make sure the plot is contained
        autosize=True,
    )
    
    return fig


def create_static_html(fig, folder_path):
    """
    Create a static HTML version of the dashboard suitable for PDF conversion
    """
    # Save plots as static images
    img_bytes = fig.to_image(format="png", width=1600, height=1200)
    img_base64 = base64.b64encode(img_bytes).decode('utf-8')
    
    # Create HTML template with embedded image
    html_template = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body { margin: 0; padding: 20px; }
            .dashboard-container { width: 100%; }
            img { width: 100%; height: auto; }
        </style>
    </head>
    <body>
        <div class="dashboard-container">
            <h1 style="text-align: center;">Mental Workload Analysis Dashboard</h1>
            <img src="data:image/png;base64,{{ image_data }}" alt="Dashboard">
        </div>
    </body>
    </html>
    """
    
    template = Template(html_template)
    static_html = template.render(image_data=img_base64)
    
    # Save static HTML
    static_html_path = os.path.join(folder_path, 'dashboard_static.html')
    with open(static_html_path, 'w', encoding='utf-8') as f:
        f.write(static_html)
    
    return static_html_path

def save_dashboard_as_pdf(folder_path, html_path, pdf_path):
    """
    Save the dashboard as both HTML and PDF
    
    Parameters:
    folder_path (str): Base folder path
    html_path (str): Path to save the interactive HTML file
    pdf_path (str): Path to save the PDF file
    """
    # Create the dashboard
    fig = create_dashboard(folder_path)
    
    # Save interactive HTML
    fig.write_html(html_path)
    
    # Create and save static version for PDF
    static_html_path = create_static_html(fig, folder_path)
    
    # Configure PDF options
    options = {
        'page-size': 'A3',
        'orientation': 'Landscape',
        'margin-top': '0.75in',
        'margin-right': '0.75in',
        'margin-bottom': '0.75in',
        'margin-left': '0.75in',
        'encoding': 'UTF-8',
        'no-outline': None,
        'enable-local-file-access': None
    }
    
    try:
        pdfkit.from_file(static_html_path, pdf_path, options=options)
        print(f"Dashboard saved as PDF: {pdf_path}")
        
        # Clean up static HTML file
        os.remove(static_html_path)
        
    except Exception as e:
        print(f"Error converting to PDF: {str(e)}")
        print("Please ensure wkhtmltopdf is installed on your system.")


# Run the dashboard creation
if __name__ == "__main__":
    folder_path = "ASM"
    fig = create_dashboard(folder_path)
    # Save the dashboard
    # fig.write_html(os.path.join(folder_path, 'dashboard.html'))
    # Also display it
    # fig.show()
    # Instead of just fig.show(), use the scrollable container
    show_dashboard(fig)
    
    # Still save the full dashboard to file
    # fig.write_html(os.path.join(folder_path, 'dashboard.html'))
    html_path = os.path.join(folder_path, 'dashboard.html')
    pdf_path = os.path.join(folder_path, 'dashboard.pdf')
    save_dashboard_as_pdf(folder_path, html_path, pdf_path)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
